<a href="https://colab.research.google.com/github/clemsage/NeuralDocumentClassification/blob/master/skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the computing environment


## Install and import PyTorch

Select "GPU" in the Accelerator drop-down on Notebook Settings through the Edit menu.

In [1]:
# %pip install torch torchvision numpy matplotlib Pillow datasets
import torch

print(torch.__version__)

2.4.1+cu121


## Confirm PyTorch can see the GPU

In [2]:
print(torch.cuda.is_available())

False


## Additional information about hardware

For CPU information and RAM, run:

In [3]:
!cat /proc/cpuinfo
!cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 154
model name	: 12th Gen Intel(R) Core(TM) i7-12800H
stepping	: 3
microcode	: 0xffffffff
cpu MHz		: 2803.212
cache size	: 24576 KB
physical id	: 0
siblings	: 20
core id		: 0
cpu cores	: 10
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 21
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology tsc_reliable nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch ssbd ibrs ibpb stibp ibrs_enhanced fsgsbase bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 xsaves umip gfni vaes vpclmulqdq rdpid fsrm md_clear flush_l1d arch_capabilities
bugs		: spectre_v1 spectre_v2 spec_store_bypass swapgs retbleed eibrs_pbrsb
bogomips	: 5606.42
clflush size	

MemTotal:       32708976 kB
MemFree:        28445604 kB
MemAvailable:   28878440 kB
Buffers:           21196 kB
Cached:           700264 kB
SwapCached:         7356 kB
Active:           257032 kB
Inactive:        3397928 kB
Active(anon):       3316 kB
Inactive(anon):  2931968 kB
Active(file):     253716 kB
Inactive(file):   465960 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:       8388608 kB
SwapFree:        8278428 kB
Dirty:               148 kB
Writeback:             0 kB
AnonPages:       2830996 kB
Mapped:           418724 kB
Shmem:              1808 kB
KReclaimable:     145644 kB
Slab:             223188 kB
SReclaimable:     145644 kB
SUnreclaim:        77544 kB
KernelStack:       12080 kB
PageTables:        46532 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    24743096 kB
Committed_AS:    3930684 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       34292 kB
VmallocChunk:          0 kB
Percpu:          

## Other useful package imports

In [3]:
import importlib
import operator
import os
import pickle
import sys
from collections import Counter
from dataclasses import dataclass
from functools import reduce
from os import path

import matplotlib.pyplot as plt
import numpy as np
import tqdm


# Working on the dataset

The dataset is a subset of the [RVL-CDIP dataset](https://www.cs.cmu.edu/~aharley/rvl-cdip/). See [Harley et al.](http://scs.ryerson.ca/~aharley/icdar15/harley_convnet_icdar15.pdf) and [Asim et al.](https://www.dfki.de/fileadmin/user_upload/import/10637_Asim_Document_Image_Classification.pdf) papers for recent works on this dataset.

## Information about the dataset

This project only considers the following 5 classes among the 16 classes of the original dataset:

In [4]:
class_names = ["email", "form", "handwritten", "invoice", "advertisement"]
NUM_CLASSES = len(class_names)

## Import the dataset

First, clone or pull the GitHub repository of the project:

In [6]:
if not os.path.exists("NeuralDocumentClassification"):
    !git clone https://github.com/thibaultdouzon/NeuralDocumentClassification.git
else:
    !git -C NeuralDocumentClassification pull
sys.path.append("NeuralDocumentClassification")

Cloning into 'NeuralDocumentClassification'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 206 (delta 10), reused 24 (delta 8), pack-reused 177 (from 1)
Receiving objects: 100% (206/206), 456.24 KiB | 4.96 MiB/s, done.
Resolving deltas: 100% (101/101), done.


Download the train, test and validation dataset assignments from this [Google Drive](https://drive.google.com/drive/folders/1Pkd6sUkDGBUymWKK93abZx1MQiWmzFgP):

In [6]:
from src import download_dataset

importlib.reload(download_dataset)
dataset_path = "dataset"

download_dataset.download_and_extract("all", dataset_path)

Each dataset file is a binary dump that can be loaded with the [Pickle](https://docs.python.org/3.11/library/pickle.html) module.

In [7]:
with open(path.join(dataset_path, "train.pkl"), "rb") as f:
    train_dataset = pickle.load(f)

with open(path.join(dataset_path, "test.pkl"), "rb") as f:
    test_dataset = pickle.load(f)

with open(path.join(dataset_path, "validation.pkl"), "rb") as f:
    validation_dataset = pickle.load(f)


for split_name, split_dataset in zip(
    ["train", "test", "validation"], [train_dataset, test_dataset, validation_dataset]
):
    print(f"{split_name}_dataset contains {len(split_dataset)} documents")
train_dataset[0].keys()


train_dataset contains 5000 documents
test_dataset contains 1000 documents
validation_dataset contains 500 documents


dict_keys(['id', 'image', 'label', 'words', 'boxes'])

Each `dataset` object is a `list` containing multiple document information. A document is a `dict` with the following structure:

```json
{
    "id": "Unique document identifier",
    "image": "A PIL.Image object containing the document's image",
    "label": "A number between in [0 .. 4] representing the class of the document",
    "words": "A list of words extracted from the image with an OCR",
    "boxes": "A list of tuples of numbers providing the position of each word in the document"
}
```

# Explore the data

Print 5 image from the training dataset using [matplotlib](https://matplotlib.org/stable/tutorials/images.html)'s `plt` module:

In [ ]:
### Insert your code here ###
# See the expected solution by clicking on the cell below

In [ ]:
# @title
for document in train_dataset[:5]:
    print(class_names[document["label"]])
    plt.imshow(document["image"].convert("RGB"))
    plt.show()

Try to answer the following questions:

What is the shape of the images?
How are the different classes distributed?
Using subplots, show an image of each class.

# Creating Pytorch datasets and dataloaders for Computer Vision task

The first goal of this section is to create `torch.utils.data.Dataset` for the classification task using only the image of the document.

We will define a class inheriting [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) called `DocumentImageDataset`.

It should be able to create an instance of `DocumentImageDataset` using our previously loaded datasets.
For simplification, all images should be formatted to black and white (remove the color channel), and resized to a fixed (512, 512) size. Use [`torchvision.transforms.v2.functional`](https://pytorch.org/vision/main/transforms.html#v2-api-reference-recommended) module to convert a `PIL.Image` to a `torch.Tensor` and perform the simplifications.

Upon iteration, it should return an `ImageSample` object defined as follows:



In [8]:
import torch
import torch.utils.data as data
import torchvision.transforms.v2.functional as F


@dataclass
class ImageSample:
    image: torch.Tensor  # shape: (H, W)
    label: int  # 0 ≤ label < NUM_CLASSES

    def __post_init__(self):
        "Some assertions to check the validity of the data"
        assert self.image.shape == (
            512,
            512,
        ), f"Expected shape (512, 512), got {self.image.shape}"
        assert torch.all(self.image <= 1.0) and torch.all(
            self.image >= 0.0
        ), "Expected each pixel of image in range [0.0, 1.0]"
        assert self.label in range(
            NUM_CLASSES
        ), f"Expected label in range [0 .. {NUM_CLASSES-1}], got {self.label}"

In [ ]:
# Fill the methods of the class DocumentImageDataset


class DocumentImageDataset(data.Dataset):
    def __init__(self, dataset: list[dict]):
        self.dataset = dataset
        raise NotImplementedError

    def __len__(self) -> int:
        """This method returns the length of the dataset"""
        raise NotImplementedError

    def __getitem__(self, idx: int) -> ImageSample:
        """This method returns the idx-th sample of the dataset
        If idx is out of bounds, it should raise an IndexError"""

        raise NotImplementedError

In [9]:
# @title


class DocumentImageDataset(data.Dataset):
    def __init__(self, dataset: list[dict]):
        self.dataset = dataset

    def __len__(self) -> int:
        """This method returns the length of the dataset"""
        return len(self.dataset)

    def __getitem__(self, idx: int) -> ImageSample:
        """This method returns the idx-th sample of the dataset
        If idx is out of bounds, it should raise an IndexError"""

        return ImageSample(
            image=F.to_dtype(
                F.to_image(F.resize(self.dataset[idx]["image"], size=[512, 512])),
                dtype=torch.float32,
                scale=True,
            ),
            label=self.dataset[idx]["label"],
        )

If your implementation is correct, you should be able to create an instance of `DocumentImageDataset` and get its 0th element without error

In [14]:
image_dataset = DocumentImageDataset(validation_dataset)
image_dataset[0]  # no error here

ImageSample(image=tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), label=3)

The final goal of this section is to implement a [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) that wraps the `DocumentImageDataset` and handles useful tasks like shuffling and batching.

No need to create a new class, we simply need to implement the `collate_fn` that takes a list of `ImageSample` and should return an `ImageBatch`.


hint: Use `torch.tensor` and `torch.stack` to respectively convert a python list to a `torch.Tensor` and stack multiple tensors together into a new one along a new dimension.

In [10]:
@dataclass
class ImageBatch:
    images: torch.Tensor
    labels: torch.Tensor

    def __post_init__(self):
        assert self.images.shape[0] == self.labels.shape[0]
        assert self.images.shape[1:] == (512, 512)
        assert len(self.images.shape) == 3
        assert len(self.labels.shape) == 1

In [11]:
def collate_fn(batch: list[ImageSample]) -> ImageBatch:
    """This function should return a batch of samples as an ImageBatch object"""
    raise NotImplementedError

In [12]:
# @title


def collate_fn(batch: list[ImageSample]) -> ImageBatch:
    """This function should return a batch of samples as an ImageBatch object"""
    return ImageBatch(
        images=torch.stack(
            [sample.image for sample in batch], dim=0
        ),  # shape: (B, H, W)
        labels=torch.tensor([sample.label for sample in batch]),  # shape: (B,)
    )

If your implementation is correct, you should be able to create a dataloader with a batch size and retrieve the first batch.

In [15]:
dataloader = data.DataLoader(
    image_dataset, batch_size=5, collate_fn=collate_fn, shuffle=True, drop_last=True
)
next(iter(dataloader))  # no error here

ImageBatch(images=tensor([[[0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[0.0000, 0.0000, 0.0196,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.1333,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.1882,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.0000, 0.0118, 0.3529,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0235, 0.4431,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 0.0431, 0.5765,  ..., 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 

# Visual classifiers

In [16]:
from torch import nn

## Multi Layer Perceptron

### Set up the layers

Build a neural network composed of one fully connected (aka dense, or `Linear` in torch) hidden layer with 128 [ReLu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) units.

Each image must be flattened to a single (512 × 512) dimension before being fed to the linear layer.

Use `torch.nn` (nn stands for Neural Network) module for all those operations.

In [ ]:
mlp_model = nn.Sequential(
    # Fill the layers of the model
    # It should take an input of shape (B, 512, 512)
    # and output a tensor of shape (B, NUM_CLASSES)
)

In [38]:
# @title
mlp_model = nn.Sequential(
    nn.Flatten(start_dim=1),  # Do not flatten the batch dimension
    nn.Linear(512 * 512, 128),  # d_input = n_pixels in an image = 512 × 512
    nn.ReLU(),
    nn.Linear(128, NUM_CLASSES),  # d_output = NUM_CLASSES
)

conv_model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(start_dim=1),
    nn.Linear(128 * 128, NUM_CLASSES),
)

In [39]:
print(
    f"Your model uses {sum([reduce(operator.mul, p.shape, 1) for p in mlp_model.parameters()]):_} trainable parameters"
)

print(
    f"Your model uses {sum([reduce(operator.mul, p.shape, 1) for p in conv_model.parameters()]):_} trainable parameters"
)

Your model uses 33_555_205 trainable parameters
Your model uses 86_725 trainable parameters


### Train the model

Pytorch does not provide a ready to use training loop function like Tensorflow does.
We will implement it ourselves.

We must first implement the training over a full iteration over the dataloader.
It will take the model, the dataloader, a loss function, an optimizer and a device to run on.

hint: help yourselves with the torch [documentation](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html#the-training-loop)


In [ ]:
def train_one_epoch(
    model: nn.Module,
    dataloader: data.DataLoader,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,  # type: ignore
    device: torch.device,
) -> float:
    """This function should train the model for one epoch and return the average loss"""

    raise NotImplementedError

In [33]:
# @title
def train_one_epoch(
    model: nn.Module,
    dataloader: data.DataLoader,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,  # type: ignore
    device: torch.device,
) -> float:
    """This function should train the model for one epoch and return the average loss"""
    model.train()
    model.to(device)

    epoch_loss = 0.0
    with tqdm.tqdm(desc="Training", total=len(dataloader)) as pbar:
        for i, batch in enumerate(dataloader):
            images, labels = batch.images.to(device), batch.labels.to(device)

            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Compute model's predictions
            loss = loss_fn(outputs, labels)  # Compute the loss

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            pbar.set_postfix(loss=epoch_loss / (i + 1))
            pbar.update(1)
    mean_loss = epoch_loss / len(dataloader)
    print(f"Training loss: {mean_loss:.4f}")
    return mean_loss

We also need to implement an evaluation method that evaluates the model's performance on a test or validation set.

It might compute the average loss and performance metric that we will use to compare models.

In [ ]:
def evaluate(
    model: nn.Module,
    dataloader: data.DataLoader,
    loss_fn: nn.Module,
    metric_fn: nn.Module,
    device: torch.device,
) -> tuple[float, float]:
    """This function should evaluate the model on the dataset and return the average loss and metric"""

    raise NotImplementedError

In [34]:
def evaluate(
    model: nn.Module,
    dataloader: data.DataLoader,
    loss_fn: nn.Module,
    metric_fn: nn.Module,
    device: torch.device,
) -> tuple[float, float]:
    """This function should evaluate the model on the dataset and return the average loss and metric"""
    model.eval()
    model.to(device)

    epoch_loss = 0.0
    epoch_metric = 0.0
    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader, desc="Evaluation"):
            images, labels = batch.images.to(device), batch.labels.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, labels)
            metric = metric_fn(outputs.argmax(dim=-1), labels)

            epoch_loss += loss.item()
            epoch_metric += metric.item()

        mean_loss = epoch_loss / len(dataloader)
        mean_metric = epoch_metric / len(dataloader)
        print(f"Evaluation loss: {mean_loss:.4f}")
        print(f"Evaluation metric: {mean_metric:.4f}")
        return mean_loss, mean_metric


Let's now implement the outer loop that trains the model over several epochs.

After each epoch, we want to control the model's performance on the validation set.

More confisticated training procedures might include model savings, modifying the learning rate or reporting to a dashboard.

In [ ]:
def train(
    model: nn.Module,
    train_dataloader: data.DataLoader,
    validation_dataloader: data.DataLoader,
    loss_fn: nn.Module,
    metric_fn: nn.Module,
    optimizer: torch.optim.Optimizer,  # type: ignore
    device: torch.device,
    n_epochs: int = 10,
) -> tuple[list[float], list[float], list[float]]:
    """This function should train the model for 10 epochs and return the training and validation losses and metrics"""
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1}/{n_epochs}")
        # Train the model here

        # Evaluate the model here
    raise NotImplementedError


In [35]:
# @title


def train(
    model: nn.Module,
    train_dataloader: data.DataLoader,
    validation_dataloader: data.DataLoader,
    loss_fn: nn.Module,
    metric_fn: nn.Module,
    optimizer: torch.optim.Optimizer,  # type: ignore
    device: torch.device,
    n_epochs: int = 10,
) -> tuple[list[float], list[float], list[float]]:
    """This function should train the model for some epochs and return the training and validation losses"""
    train_losses = []
    validation_losses = []
    validation_metrics = []

    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1}/{n_epochs}")
        train_loss = train_one_epoch(
            model, train_dataloader, loss_fn, optimizer, device
        )
        train_losses.append(train_loss)

        validation_loss, validation_metric = evaluate(
            model, validation_dataloader, loss_fn, metric_fn, device
        )
        validation_losses.append(validation_loss)
        validation_metrics.append(validation_metric)

    return train_losses, validation_losses, validation_metrics

In [40]:
import torchmetrics

train_loader = data.DataLoader(
    DocumentImageDataset(train_dataset),
    batch_size=16,
    collate_fn=collate_fn,
    shuffle=True,
)
validation_loader = data.DataLoader(
    DocumentImageDataset(validation_dataset),
    batch_size=16,
    collate_fn=collate_fn,
    shuffle=False,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
metric_fn = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(device)

n_epochs = 5

hist = train(
    conv_model,
    train_loader,
    validation_loader,
    loss_fn,
    metric_fn,
    optimizer,
    device,
    n_epochs=n_epochs,
)

Epoch 1/5


Training:   0%|          | 0/313 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [16, 1, 3, 3], expected input[1, 16, 512, 512] to have 1 channels, but got 16 channels instead

### Evaluation on the test set

In [ ]:
# @title
model.evaluate(labeled_test_ds, verbose=2)

Are these values different from their training counterparts ?

### Prediction on the test set

Implement a function that gathers the model predictions and the ground truth labels for a random batch of a given dataset:

In [ ]:
def predict_random_batch(model, dataset):
    """
    Sample a random batch of the dataset and return the images of this batch
    as well as its labels and the predicted classes of a given model

    Parameters
    ----------
    model : tf.keras.Model
    dataset: tf.data.Dataset

    Returns
    -------
    images: np.ndarray or EagerTensor
    labels: list of str
    predicted_classes: list of str
    """
    images, labels = next(iter(dataset))

    # get label names for the sampled batch

    # make predictions
    predicted_classes = None

    return images, labels, predicted_classes

In [ ]:
# @title
def predict_random_batch(model, dataset):
    """
    Sample a random batch of the dataset and return the images of this batch
    as well as its labels and the predicted classes of a given model

    Parameters
    ----------
    model : tf.keras.Model
    dataset: tf.data.Dataset

    Returns
    -------
    images: np.ndarray or EagerTensor
    labels: list of str
    predicted_classes: list of str
    """
    images, labels = next(iter(dataset))

    # get label names for the sampled batch
    labels = [class_names[i] for i in labels]

    # make predictions
    predictions = model.predict(images)
    predicted_classes_idx = np.argmax(predictions, axis=1)
    predicted_classes = [class_names[i] for i in predicted_classes_idx]

    return images, labels, predicted_classes

Plot the first 9 images of this batch, give their labels and predicted classes in the legend:

In [ ]:
def plot_images_predictions_and_labels(images, labels, predicted_classes):
    plt.figure(figsize=(30, 40))

    for im_idx in range(9):
        plt.subplot(3, 3, im_idx + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(np.squeeze(images[im_idx]), cmap="gray")
        plt.xlabel("label: %s\npred: %s" % (labels[im_idx], predicted_classes[im_idx]))

    plt.show()


result = predict_random_batch(model, labeled_test_ds)
plot_images_predictions_and_labels(*result)

### Under the Hood

Implement an ReLu dense layer by creating its weights and biases and giving the transformation from inputs to outputs:

In [ ]:
# https://www.tensorflow.org/guide/keras/custom_layers_and_models#the_layer_class
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units, input_dim):
        super(MyDenseLayer, self).__init__()
        self.w = self.add_weight(
            shape=None,  ## Insert the shape of the weight matrix here
            initializer="glorot_uniform",  # Default initializer for weights of a tf.keras.layers.Dense layer
            trainable=True,
        )

        self.b = self.add_weight(
            shape=None,  ## Insert the shape of the bias vector here
            initializer="zeros",  # Default initializer for biases of a tf.keras.layers.Dense layer
            trainable=True,
        )

    def call(self, inputs):
        outputs = None
        return outputs

In [ ]:
# @title
# https://www.tensorflow.org/guide/keras/custom_layers_and_models#the_layer_class
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units, input_dim):
        super(MyDenseLayer, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units),
            initializer="glorot_uniform",  # Default initializer for weights of a tf.keras.layers.Dense layer
            trainable=True,
        )

        self.b = self.add_weight(
            shape=(units,),
            initializer="zeros",  # Default initializer for biases of a tf.keras.layers.Dense layer
            trainable=True,
        )

    def call(self, inputs):
        return tf.keras.activations.relu(tf.matmul(inputs, self.w) + self.b)

Using your custom hidden layer, set up again the layers of the model defined previously:

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
        ## Insert your custom hidden layer here
        tf.keras.layers.Dense(NUM_CLASSES, activation="softmax"),
    ]
)

In [ ]:
# @title
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
        MyDenseLayer(128, IMG_HEIGHT * IMG_WIDTH),
        tf.keras.layers.Dense(NUM_CLASSES, activation="softmax"),
    ]
)

Lower-level implementation of the model compile step:

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")


@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

Lower-level implementation of the model fit step:

In [ ]:
for epoch in range(EPOCHS):
    for images, labels in labeled_train_ds:
        train_step(images, labels)
        template = "Epoch {}, Loss: {}, Accuracy: {}"
        print(
            template.format(
                epoch + 1, train_loss.result(), train_accuracy.result() * 100
            )
        )

## Convolutional Neural Networks (CNN)

### Training from scratch

Create and compile a model alterning convolution and max pooling layers. You can add some fully connected layers between the last locally connected layer and the output layer. Start with a shallow network (4 or 5 convolution layers) and progressively move to deeper architectures: 

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

model = tf.keras.Sequential(
    [
        # Alterning Conv2D and MaxPooling2D layers
        # Some dense hidden layer(s)
        Dense(NUM_CLASSES, activation="softmax")
    ]
)

print(model.summary())

model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
# @title
from tensorflow.keras.layers import Dense

shallow_model = tf.keras.Sequential(
    [
        Conv2D(
            16,
            3,
            padding="same",
            activation="relu",
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 1),
        ),
        MaxPooling2D(4),
        Conv2D(32, 3, padding="same", activation="relu"),
        MaxPooling2D(4),
        Conv2D(64, 3, padding="same", activation="relu"),
        MaxPooling2D(4),
        Conv2D(128, 3, padding="same", activation="relu"),
        MaxPooling2D(4),
        Flatten(),
        Dense(128, activation="relu"),
        Dense(NUM_CLASSES, activation="softmax"),
    ]
)

deep_model = tf.keras.Sequential(
    [
        Conv2D(
            16,
            3,
            padding="same",
            activation="relu",
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 1),
        ),
        MaxPooling2D(),
        Conv2D(32, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Conv2D(64, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Conv2D(128, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Conv2D(256, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Conv2D(256, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Conv2D(256, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Conv2D(256, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Flatten(),
        Dense(256, activation="relu"),
        Dense(NUM_CLASSES, activation="softmax"),
    ]
)

model_with_strides = tf.keras.Sequential(
    [
        Conv2D(
            16,
            3,
            padding="same",
            activation="relu",
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 1),
        ),
        MaxPooling2D(),
        Conv2D(32, 3, padding="same", activation="relu"),
        MaxPooling2D(),
        Conv2D(64, 3, padding="same", activation="relu", strides=2),
        MaxPooling2D(),
        Conv2D(128, 3, padding="same", activation="relu", strides=2),
        MaxPooling2D(),
        Conv2D(128, 3, padding="same", activation="relu", strides=2),
        MaxPooling2D(),
        Flatten(),
        Dense(128, activation="relu"),
        Dense(NUM_CLASSES, activation="softmax"),
    ]
)
model = deep_model
print(model.summary())

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

Fit the CNN on the training data:

In [ ]:
EPOCHS = 10
model.fit(labeled_train_ds, epochs=EPOCHS)

Evaluate the trained model on the test set:

In [ ]:
model.evaluate(labeled_test_ds, verbose=2)

You should reach test accuracy greater than 0.99 !




Plot images, predictions and labels for some test documents:

In [ ]:
plot_images_predictions_and_labels(*predict_random_batch(model, labeled_test_ds))

### Transfer Learning with pre-trained models 

The objective is to leverage the knowledge learnt by a pre-trained image classifier. See [TensorFlow Hub](https://tfhub.dev/) to browse available state-of-the art models such as [Inception V3](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Szegedy_Rethinking_the_Inception_CVPR_2016_paper.pdf) or [MobileNet V2](https://arxiv.org/pdf/1801.04381.pdf).

Choose a pre-trained model for extracting high level feature vectors of document images:

In [ ]:
extractor_model = "inception_v3"
if extractor_model == "inception_v3":
    feature_extraction_url = (
        "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
    )
    IMG_HEIGHT, IMG_WIDTH = None, None  ## Insert expected input image shape here
elif extractor_model == "mobilenet_v2":
    feature_extraction_url = (
        "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
    )
    IMG_HEIGHT, IMG_WIDTH = None, None  ## Insert expected input image shape here

In [ ]:
# @title
extractor_model = "inception_v3"
if extractor_model == "inception_v3":
    feature_extraction_url = (
        "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
    )
    IMG_HEIGHT, IMG_WIDTH = 299, 299
elif extractor_model == "mobilenet_v2":
    feature_extraction_url = (
        "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
    )
    IMG_HEIGHT, IMG_WIDTH = 224, 224

Reshape images to the format expected by the chosen model, i.e. IMG_HEIGHT x IMG_WIDTH x 3 (RGB) and recreate training dataset:

In [ ]:
def decode_img(img):
    # convert the compressed string to a uint8 tensor
    img = tf.io.decode_png(img, channels=1)
    # convert to floats in the [0,1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    # convert to RGB color scale
    img = tf.concat([img for _ in range(3)], axis=-1)  # R = G = B
    return img


def process_path(file_path):
    label = labels_idx.lookup(file_path)

    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label


labeled_train_ds = list_train_ds.map(
    process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE
)
labeled_train_ds = labeled_train_ds.cache("/tmp/%dx%dx3" % (IMG_HEIGHT, IMG_WIDTH))
labeled_train_ds = labeled_train_ds.shuffle(2048).batch(batch_size)
labeled_train_ds = labeled_train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Construct our own image classifier by retrieving and freezing the hidden layers of the pre-trained model: 

In [ ]:
import tensorflow_hub as hub


In [ ]:
# @title


Train this new model:

In [ ]:
EPOCHS = 10
model.fit(labeled_train_ds, epochs=EPOCHS)